In [ ]:
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base = automap_base() 
Base.prepare(engine, reflect=True)
Base.classes.keys()

In [ ]:
measurement = Base.classes.measurement
station = Base.classes.station
session = Session(engine)

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/<start>"
        f"/api/v1.0/<start>/<end>"
    )

In [ ]:
@app.route("/api/v1.0/precipitation")
def precipitation():
    session = Session(engine)
    results = session.query(measurement.date, measurement.prcp).all()
    session.close()
    
    all_precipitation = []
    for date, prcp in results:
        precipitation_dict = {}
        precipitation_dict["Date"] = date
        precipitation_dict["Precipitation"] = prcp
        all_precipitation.append(precipitation_dict)

    return jsonify(all_precipitation)

In [ ]:
@app.route("/api/v1.0/stations")
def stations():
    session = Session(engine)
    results = session.query(measurement.station).all()
    session.close()
    all_stations = list(np.ravel(results))
    return jsonify(all_stations)

In [ ]:
@app.route("/api/v1.0/tobs")
def tobs():
    session = Session(engine)
    results = session.query(measurement.date, measurement.tobs).\
        filter(measurement.station == 'USC00519281').\
        filter(measurement.date <= '2017-08-23').\
        filter(measurement.date >= '2016-08-24').all()
    session.close()
    
    all_tobs = []
    for date, tobs in results:
        tobs_dict = {}
        tobs['Date'] = date
        tobs['Temperature'] = tobs
        all_tobs.append(tobs_dict)
    
    return jsonify(all_tobs)

In [ ]:
@app.route("/api/v1.0/bydate/<start_date>")
def bydate(start_date):
    start_date= datetime.strptime('%Y-%m-%d')
    session = Session(engine)
    results = session.query(func.min(measurement.tobs), func.max(measurement.tobs), func.avg(measurement.tobs)).\
        filter(start_date <= '2017-08-23').all()
    session.close()
    after_date = list(np.ravel(results))
    return jsonify(after_date)

In [ ]:
@app.route("/api/v1.0/bydate/<start_date>/<end_date>")
def bydates(start_date, end_date):
    start_date= datetime.strptime('%Y-%m-%d')
    end_date= datetime.strptime('%Y-%m-%d')
    session = Session(engine)
    results = session.query(func.min(measurement.tobs), func.max(measurement.tobs), func.avg(measurement.tobs)).\
        filter(start_date <= '2017-08-23').\
        filter(end_date >= '2016-08-24').all()
    session.close()
    btwn_dates = list(np.ravel(results))
    return jsonify(btwn_dates)

In [ ]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)